In [3]:
##For Data consideration see Kaggle data tab.
##Predict for 
#id,date,store_nbr,item_nbr,onpromotion
#125497040,2017-08-16,1,96995,False
#...
#128867503,2017-08-31,54,2134244,False


#Inspired on Kaggle
#(https://www.kaggle.com/tarobxl/how-the-test-set-is-split-lb-0-532)

#import a dataset
import pandas as pd
from datetime import timedelta

dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8'}

train = pd.read_csv('/Users/Koos/Downloads/DataFav/train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 101688779) #Skip dates before 2017-01-01
                    )

In [4]:
train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek # dow is day of week - KtB


In [6]:
# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
)

In [8]:
train

unit_sales  dow
date       store_nbr item_nbr                 
2016-12-31 54        2048674     0.693147  5.0
                     99197            NaN  NaN
                     103665           NaN  NaN
                     105574           NaN  NaN
                     105857           NaN  NaN
                     106716           NaN  NaN
                     108698           NaN  NaN
                     108786           NaN  NaN
                     108797           NaN  NaN
                     108862           NaN  NaN
                     108952           NaN  NaN
                     114790           NaN  NaN
                     114800           NaN  NaN
                     115267           NaN  NaN
                     115693           NaN  NaN
                     115720           NaN  NaN
                     115850           NaN  NaN
                     115891           NaN  NaN
                     115892           NaN  NaN
                     115893           NaN  NaN
                     115894           NaN  NaN
                     119026           NaN  NaN
                     119624           NaN  NaN
                     121964           NaN  NaN
                     122095           NaN  NaN
                     123347           NaN  NaN
                     123927           NaN  NaN
                     127547           NaN  NaN
                     129635           NaN  NaN
                     153078           NaN  NaN
...                                   ...  ...
2017-08-15 52        2122676     0.693147  1.0
                     2120723          NaN  NaN
                     2122947          NaN  NaN
                     2116238     0.693147  1.0
                     2123468     0.693147  1.0
                     2123209          NaN  NaN
                     2116132          NaN  NaN
                     2121610          NaN  NaN
                     2121690          NaN  NaN
                     2122868          NaN  NaN
                     2116139          NaN  NaN
                     2123750          NaN  NaN
                     2123711          NaN  NaN
                     2123859          NaN  NaN
                     2114752          NaN  NaN
                     2123863          NaN  NaN
                     2123790          NaN  NaN
                     2123747          NaN  NaN
                     2011437          NaN  NaN
                     2011457          NaN  NaN
                     2011470          NaN  NaN
                     2123463          NaN  NaN
                     2126842          NaN  NaN
                     2011448          NaN  NaN
                     2011468          NaN  NaN
                     2122818          NaN  NaN
                     2011459          NaN  NaN
                     2126944          NaN  NaN
                     2123839          NaN  NaN
                     2011451          NaN  NaN

[49469616 rows x 2 columns]

In [21]:
#clean up
del u_dates, u_stores, u_items

In [9]:
# Fill NaNs
train.loc[:, 'unit_sales'].fillna(0, inplace=True)
train.reset_index(inplace=True) # reset index and restoring unique columns  
lastdate = train.iloc[train.shape[0]-1].date

In [13]:
test = pd.read_csv('/Users/Koos/Downloads/DataFav/test.csv', usecols=[0,1,2,3,4], dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek

In [14]:
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')
ma_dw.reset_index(inplace=True)
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')
ma_wk.reset_index(inplace=True)

In [18]:
ma_wk.head()

,store_nbr,item_nbr,mawk
0,1,96995,0.844024
1,1,99197,1.016067
2,1,103520,1.083762
3,1,103665,1.289606
4,1,105574,1.814829


In [19]:
#Moving Averages
#.to_frame is a renaming function of pandas

ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')
for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

del tmp,tmpg